<a href="https://colab.research.google.com/github/sahilsait/credit-risk-assessment-using-GNNs/blob/main/data_preprocessing_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('/content/drive/MyDrive/datasets/dataset.csv')
data.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,conm,curcd,fyr,...,lct,lt,ni,oancf,oiadp,oibdp,sale,xint,costat,sic
0,1004,2014-05-31,2013,INDL,C,D,STD,AAR CORP,USD,5,...,402.1,1198.8,72.9,139.8,142.6,256.0,2035.0,42.0,A,5080
1,1004,2015-05-31,2014,INDL,C,D,STD,AAR CORP,USD,5,...,412.0,669.9,10.2,-43.0,-8.6,83.7,1594.3,26.5,A,5080
2,1004,2016-05-31,2015,INDL,C,D,STD,AAR CORP,USD,5,...,329.0,576.3,47.7,32.1,66.1,136.9,1662.6,6.4,A,5080
3,1004,2017-05-31,2016,INDL,C,D,STD,AAR CORP,USD,5,...,335.1,589.9,56.5,21.8,77.2,148.2,1767.6,5.5,A,5080
4,1004,2018-05-31,2017,INDL,C,D,STD,AAR CORP,USD,5,...,333.3,588.4,15.6,64.3,86.0,126.5,1748.3,8.0,A,5080


### Basic cleaning

In [ ]:
# Remove zero or negative asset
data = data[data['at'] > 0]

# Remove zero or negative sales
data = data[data['sale'] > 0]

### Filtering for SME companies based on paper's criteria

In [ ]:
# Calculate asset threshold for each year (80th percentile)
yearly_thresholds = data.groupby('fyear')['at'].quantile(0.8)

# Create mask for SME filtering
sme_mask = data.apply(lambda x: x['at'] < yearly_thresholds[x['fyear']], axis=1)

# Apply filter
sme_data = data[sme_mask]

print(f"Companies before SME filtering: {data['gvkey'].nunique()}")
print(f"Companies after SME filtering: {sme_data['gvkey'].nunique()}")

Companies before SME filtering: 6717
Companies after SME filtering: 5902


### Calculate all required financial ratios

In [ ]:
# Create copy to avoid modifying original
ratio_data = data.copy()

# 1. Debt Structure Ratios
ratio_data['debt_total'] = ratio_data['dltt'] + ratio_data['dlc']  # Total debt
ratio_data['short_term_debt_ratio'] = ratio_data['dlc'] / ratio_data['debt_total']  # Short-term debt ratio
ratio_data['interest_coverage'] = ratio_data['oibdp'] / ratio_data['xint']  # Interest coverage

# 2. Profitability Ratios
ratio_data['roa'] = ratio_data['ni'] / ratio_data['at']  # Return on assets
ratio_data['roe'] = ratio_data['ni'] / ratio_data['ceq']  # Return on equity
ratio_data['ebitda_margin'] = ratio_data['oibdp'] / ratio_data['sale']  # EBITDA margin

# 3. Liquidity Ratios
ratio_data['current_ratio'] = ratio_data['act'] / ratio_data['lct']  # Current ratio
ratio_data['quick_ratio'] = (ratio_data['act'] - ratio_data['invt']) / ratio_data['lct']  # Quick ratio
ratio_data['cash_ratio'] = ratio_data['che'] / ratio_data['lct']  # Cash ratio

# 4. Cash Flow Ratios
ratio_data['ocf_ebitda'] = ratio_data['oancf'] / ratio_data['oibdp']  # Operating cash flow to EBITDA
ratio_data['ebitda_debt'] = ratio_data['oibdp'] / ratio_data['debt_total']  # EBITDA to debt

# 5. Growth Rates (by company)
for company in ratio_data['gvkey'].unique():
  mask = ratio_data['gvkey'] == company
  # Revenue growth
  ratio_data.loc[mask, 'revenue_growth'] = ratio_data.loc[mask, 'sale'].pct_change()
  # Asset growth
  ratio_data.loc[mask, 'asset_growth'] = ratio_data.loc[mask, 'at'].pct_change()

### Handling outliers using winsorization

In [ ]:
# Define ratio columns
ratio_columns = [
    'short_term_debt_ratio', 'interest_coverage', 'roa', 'roe', 'ebitda_margin',
    'current_ratio', 'quick_ratio', 'cash_ratio', 'ocf_ebitda', 'ebitda_debt',
    'revenue_growth', 'asset_growth'
]

clean_data = ratio_data.copy()
# Winsorize ratios at 1st and 99th percentiles
for col in ratio_columns:
  p01 = clean_data[col].quantile(0.01)
  p99 = clean_data[col].quantile(0.99)
  clean_data[col] = clean_data[col].clip(p01, p99)

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


### Standardizing all numerical features

In [ ]:
# Get all numeric columns
numeric_cols = clean_data.select_dtypes(include=[np.number]).columns

# Remove identifier columns
cols_to_scale = [col for col in numeric_cols if col not in ['gvkey', 'fyear', 'sic', 'conm']]

final_data = clean_data.copy()

for col in cols_to_scale:
  final_data[col] = final_data[col].replace([np.inf, -np.inf], np.nan)
  col_median = final_data[col].median()

  # Replace NaN with median
  nan_count = final_data[col].isna().sum()
  final_data[col] = final_data[col].fillna(col_median)

  print(f"{col}: replaced {nan_count} NaN/infinite values with median ({col_median:.4f})")

# Initialize scaler
scaler = StandardScaler()
# Scale features
final_data[cols_to_scale] = scaler.fit_transform(final_data[cols_to_scale])

final_data.head()

fyr: replaced 0 NaN/infinite values with median (12.0000)
act: replaced 10367 NaN/infinite values with median (274.1960)
at: replaced 0 NaN/infinite values with median (1110.2975)
ceq: replaced 99 NaN/infinite values with median (309.3940)
che: replaced 4 NaN/infinite values with median (89.8595)
dlc: replaced 25 NaN/infinite values with median (10.0000)
dltt: replaced 139 NaN/infinite values with median (119.1240)
fincf: replaced 220 NaN/infinite values with median (0.0670)
invt: replaced 659 NaN/infinite values with median (5.8770)
ivncf: replaced 221 NaN/infinite values with median (-36.9830)
lct: replaced 10325 NaN/infinite values with median (128.1240)
lt: replaced 71 NaN/infinite values with median (619.6630)
ni: replaced 1 NaN/infinite values with median (11.0060)
oancf: replaced 210 NaN/infinite values with median (40.1085)
oiadp: replaced 0 NaN/infinite values with median (33.1975)
oibdp: replaced 1608 NaN/infinite values with median (53.4510)
sale: replaced 0 NaN/infinite val

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,conm,curcd,fyr,...,roa,roe,ebitda_margin,current_ratio,quick_ratio,cash_ratio,ocf_ebitda,ebitda_debt,revenue_growth,asset_growth
0,1004,2014-05-31,2013,INDL,C,D,STD,AAR CORP,USD,-1.953576,...,0.274440,0.057091,0.179690,0.075325,-0.318494,-0.427418,-0.120093,0.003111,-0.166218,-0.191924
1,1004,2015-05-31,2014,INDL,C,D,STD,AAR CORP,USD,-1.953576,...,0.239215,0.010364,0.175352,-0.076912,-0.408188,-0.459499,-0.844091,0.003868,-0.432390,-0.783552
2,1004,2016-05-31,2015,INDL,C,D,STD,AAR CORP,USD,-1.953576,...,0.274350,0.043393,0.177118,0.034499,-0.408142,-0.473162,-0.332965,0.005930,-0.190496,-0.347981
3,1004,2017-05-31,2016,INDL,C,D,STD,AAR CORP,USD,-1.953576,...,0.280335,0.048544,0.177207,0.033723,-0.436329,-0.496248,-0.392656,0.006026,-0.171552,-0.197136
4,1004,2018-05-31,2017,INDL,C,D,STD,AAR CORP,USD,-1.953576,...,0.243882,0.013889,0.176527,0.092040,-0.325019,-0.462364,-0.145911,0.004790,-0.240626,-0.245640


In [ ]:
final_data.to_csv('/content/drive/MyDrive/datasets/preprocessed_data.csv', index=False)